#### Preprocessing Data:

In [35]:
import pandas as pd
from bs4 import BeautifulSoup
#import urllib.request
import requests
#import pycountry

PATH = r'../Horse_Racing_Data/CSV_Schema/'

# ISO 2 = alpha_2='AU'
# ISO 3 = alpha_3='AUS'

#NZ NZL

pd.options.display.max_colwidth = 1000000000

horses_data =  pd.read_csv(PATH + 'horses.csv')
#horses_data =  pd.read_csv('horses.csv')
horses_data.head()

num_horses = len(horses_data)

horses_data['Country'].unique().tolist()

lookup_columns = ['Horse_ID','Horse_Name','Country']
horses_lookup = horses_data[lookup_columns]
#horses_lookup.insert(loc=3, column="URL_Suffix", value=horses_data['Horse_Name'].str.replace(" ", "-").str.lower())
horses_lookup.insert(loc=3, column="URL", value="https://www.racenet.com.au/profiles/horse/"+horses_data['Horse_Name'].str.replace(" ", "-").str.lower())
#horses_lookup.insert(loc=5, column="URL_Country_Suffix", value=horses_data['Horse_Name'].str.replace(" ", "-").str.lower()+"-"+horses_data['Country'].str.lower())
horses_lookup.insert(loc=4, column="URL_Country", value="https://www.racenet.com.au/profiles/horse/"+horses_data['Horse_Name'].str.replace(" ", "-").str.lower()+"-"+horses_data['Country'].str.lower())
horses_lookup

horses_lookup.head()

,Horse_ID,Horse_Name,Country,URL,URL_Country
0,1,SUPER AXIOM,AUS,https://www.racenet.com.au/profiles/horse/super-axiom,https://www.racenet.com.au/profiles/horse/super-axiom-aus
1,2,SEA ELITE,AUS,https://www.racenet.com.au/profiles/horse/sea-elite,https://www.racenet.com.au/profiles/horse/sea-elite-aus
2,3,LITTLE PLAYER,NZ,https://www.racenet.com.au/profiles/horse/little-player,https://www.racenet.com.au/profiles/horse/little-player-nz
3,4,NEW FUTURE,AUS,https://www.racenet.com.au/profiles/horse/new-future,https://www.racenet.com.au/profiles/horse/new-future-aus
4,5,RED MAJESTY,NZ,https://www.racenet.com.au/profiles/horse/red-majesty,https://www.racenet.com.au/profiles/horse/red-majesty-nz


#### Foal Date to DataFrame:

In [36]:
my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",  "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8"}

#horses_lookup = horses_lookup[:5]
num_horses = len(horses_lookup)

for i, row in horses_lookup.iterrows():
#    url = 'https://www.racenet.com.au/profiles/horse/super-axiom-aus' #horses_lookup['URL']
    url_ctry = row['URL_Country']
    html_content_ctry = requests.get(url_ctry, headers = my_headers).text
    soup_ctry = BeautifulSoup(html_content_ctry, 'lxml')
    result_list_ctry = len(soup_ctry.find_all('div',{'class':'profile-info__item--value'}))
    if result_list_ctry >= 5:
        foal_date = soup_ctry.find_all('div',{'class':'profile-info__item--value'})[5].find_all('span')[0].contents
        if not foal_date:
            continue #if foal_date not found, then begin next iteration with other link
        horses_lookup.at[i, 'Foal_Date'] = str(foal_date).replace('[', '').replace(']', '').replace("'", "")
        horses_lookup.at[i, 'success_flag'] = 1
        print(row['Horse_Name']+': Success')
    else:
        url = row['URL']
        html_content = requests.get(url, headers = my_headers).text
        soup = BeautifulSoup(html_content, 'lxml')
        result_list = len(soup.find_all('div',{'class':'profile-info__item--value'}))
        if result_list >= 5:
            foal_date = soup.find_all('div',{'class':'profile-info__item--value'})[5].find_all('span')[0].contents
            if not foal_date:
                continue #if foal_date not found, then begin next iteration with other link
            horses_lookup.at[i, 'Foal_Date'] = str(foal_date).replace('[', '').replace(']', '').replace("'", "")
            horses_lookup.at[i, 'success_flag'] = 1
            print(row['Horse_Name']+': Success')
        else:
            horses_lookup.at[i, 'success_flag'] = 0
            print(row['Horse_Name']+"\033[1m"+': Failed'+"\033[0m")
#return horses_lookup
            
# check coverage of Foal Date after Web Scraping:
num_successes = horses_lookup['success_flag'].sum()
num_horses = len(horses_lookup)          
success_rate = (num_successes / num_horses)

print(success_rate)

horses_lookup

C:\Users\USER\anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\USER\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


SUPER AXIOM: Success
SEA ELITE: Failed
LITTLE PLAYER: Success
NEW FUTURE: Success
RED MAJESTY: Success
VICTORIA GENERAL: Success
PICHU: Success
SKY CHAMPION: Success
SAVAQUIN: Success
IMA SINGLE MAN: Success
HELENE WISDOM STAR: Success
AION: Failed
STRONG ALLIANCE: Failed
MAN STAR: Success
WEALTHY DELIGHT: Success
VIVA CHEF: Success
LUCKY TURBO: Success
METRO WARRIOR: Failed
PRINCE OF BON BON: Failed
MORE THAN ENOUGH: Success
TELECOM JAC: Failed
JOLLY GOOD HEART: Success
SUPER COMMANDER: Success
BLAZING BEAT: Success
REWARDING TOGETHER: Failed
JOLLY FOREVER: Success
DELIGHTFUL LAOS: Failed
HAPPY RUBY: Success
SPEED TO EXCEED: Success
KINDA COOL: Success
PRIME MINISTER: Success
GRATEFUL HEART: Success
CALIFORNIADEEPSHOT: Success
MANAGEMENT SUPREME: Success
BOND A STAR: Success
ORIENTAL AMIGO: Success
RUN THE TABLE: Success
LARSON: Success
WIND N GRASS: Failed
SMILING FACE: Success
GLORIOUS LOVER: Success
WE THE SOUTH: Success
CALIFORNIA RAD: Success
MORE THAN A HORSE: Success
BRAVE KING

SAVVY KINGMAN: Success
ZONE D: Success
UNTOLD RICHES: Success
PRECISION DREAM: Success
AUGUST MOON: Success
VIVA MAMA: Success
SURE WIN WIN: Success
PRETTY CUTE: Success
TELECOM CHEETAH: Success
SOULMATE: Success
GOOD LUCK CHARM: Success
ORION PATCH: Success
YOUNG VICTORY: Success
DAILY CHARM: Success
ABOVE ALL: Success
ALL FOR SOUTH: Failed
CZARSON: Success
CELESTIAL POWER: Success
GREEN AEON: Success
WINNING FOR ALL: Success
HAPPY QIJI: Failed
PRIVATE COUNCIL: Success
BIG BROOK: Success
EXCELLENT PROPOSAL: Success
EQUALETTA BLITZ: Success
GOLFMAN STAR: Success
ENCHANTING: Success
ENTRUSTED: Success
YOUNG PASSION: Success
HAFEET ALAIN: Success
JOYFUL FORTUNE: Failed
SIGHT SUCCESS: Success
FORTUNE MASTER: Success
CLUB LAD: Failed
STORM WARNINGS: Failed
EXCEPTIONAL NICE: Failed
POISED TO STRIKE: Success
ALL JOYFUL: Success
NOBLE BIRTH: Success
SUPER TEN: Success
INNO LEGEND: Success
ANGEL OF MY EYES: Success
GALLANT CROWN: Success
WINNING DREAMER: Success
AMAZING ROCKY: Success
TELECOM 

BEAT THE BANK: Success
MOZU ASCOT: Success
COMIN' THROUGH: Failed
PERSIAN KNIGHT: Success
LIM'S CRUISER: Failed
SIR DANCEALOT: Success
MR WISEGUY: Success
ERA KING: Success
NOBLE DESIRE: Success
WELL DRAGON: Success
NOBLE CONSORT: Success
JOYFUL CHEERS: Success
SPEED FORCE: Success
COPARTNER ERA: Success
ROBOT WARRIOR: Success
HARMONY N HOME: Success
COPARTNER RAPPORT: Success
TOUCH OF FAITH: Success
STRONG PROMISE: Success
KA YING EXCELLENT: Success
RUNNING GUY: Success
CALIFORNIA LEVEE: Failed
TAILOR MADE: Success
WINNING BREW: Success
MISSION SMART: Success
AMAZING PARADISE: Success
ALL FOR ST PAUL'S: Failed
MERIDIAN TALENT: Success
MANY GAINS: Success
LUCKY VICTOR: Success
DASHING GRACE: Success
HIGH RISE SOLDIER: Success
HOME WIN: Success
KING'S EMINENCE: Failed
AMAZING BEATS: Failed
EXTRA BRUT: Success
LIGHT MY DAY: Success
WON BALL: Success
RAKKIKYATTO: Failed
SUNSET WATCH: Success
BABY STORM: Failed
FORTUNE BRILLIANT: Success
VIGOR CHAMP: Success
PRO FIT: Success
ANONYMA: Succe

UNIQUE TREASURE: Failed
UNCLE STEVE: Success
GOLD WIN: Failed
TELECOM ROCKET: Success
KUNG FU TEA: Success
BAND OF BROTHERS: Success
TRUST ME: Success
IRON KING: Success
E STAR: Success
TAKINGUFURTHER: Success
KING'S TROOPER: Failed
DR RESPECT: Success
ALPHABET INVENTION: Success
GOOD RUNNERS WAY: Success
PRECIOUS EXPRESS: Failed
HAPPY WIN WIN: Success
KING PRINT: Failed
GLORIOUS DRAGON: Failed
FLYING TIGER HERO: Failed
TIGRE DU TERRE: Success
HAPPY FAMILY: Failed
GRANDMA WIN: Failed
EPIC: Success
WAR LORD: Success
SOLAR WAI WAI: Success
WINWIN THIRTYTHREE: Success
DOR DOR: Success
WONDERFUL DAY: Success
GOLDEN MISSION: Success
CAPTAIN FANTASTIC: Success
MAX GLENEALY: Success
VICTORY IN HAND: Success
EIGHT TRIGRAMS: Success
TRIUMPHANT LORD: Success
HAPPY BOYBOY: Success
CHICKEN DANCE: Success
VOYAGE STAR: Success
BLADE RACER: Success
LITTLE GENERAL: Success
NOT USUAL TALENT: Success
MIGHTY POWER: Success
GUN GUN WIN: Failed
VICTORIOUS SEEKER: Success
DYNASTA: Success
FAMILY LEADER: Fai

...

JOLLY MAN: Failed
THE COMMANDER: Failed
PRINCE HESTER: Failed
CHEERFUL GAIN: Failed
MOON KOON: Failed
IMPERIAL PRIDE: Failed
LOCHVALE: Failed
CO-TACK: Failed
DOCTOR KING: Failed
HOLE IN ONE: Success
TIVOLI: Success
HARVEST MOON: Success
LOVE TO WIN: Failed
WING BRASS: Failed
EXCELLENT KID: Failed
BOLD DEAL: Failed
HARMONY WEALTH: Failed
MAGIC BOW: Failed
GOOD RECORD: Failed
GOJINKA: Failed
LADY LORENZ: Failed
PALOPPO: Failed
WONDERFUL WIN: Failed
DARTSON: Failed
REVENGE: Success
FAVOURITE BOY: Failed
GOLDEN HARVEST: Success
COMMAND POST: Failed
HAPPY FELLOW: Failed
GUNSMOKE: Success
STAR VALUE: Success
WISE GUY: Failed
REAL SUCCESS: Success
PERFECTION: Success
ELECTRONIC REIGN: Failed
HOLY SALUTE: Failed
PROUD AVON: Failed
EASTERN TREASURE: Failed
BEAU JET: Success
TERRY: Success
SINCLAIR II: Failed
PROSPERITY: Success
MAGIC LUCK: Failed
GAMIN: Failed
MUN WAI: Failed
BONA FIDE: Success
FAME N'FORTUNE: Failed
CLAIR DE LUNE: Success
WORLD CHAMP: Failed
BETHLEHEM STAR: Failed
HEAVENLY STA

MR BIG: Success
WINNING SHOT: Success
EASY WAY: Failed
SAPPHIRE: Success
CAMBRONNE: Failed
OIO: Failed
FUN FUN: Failed
LIGHTNING STREAK: Success
GRACEFUL: Success
MICHAEL HARDEY: Failed
ROCKY: Success
HAPPY GATHERING: Success
GOODIE GOODIE: Success
AMERICAN SUCCESS: Failed
DISCO: Success
ALL SUPREME: Failed
HEART'S CONTENT: Failed
SUPER DRAGON: Success
FAMOUS: Success
AMITY: Success
KING'S CHOICE: Failed
CONTENDER: Success
AVANTI: Success
THE MIRACLE STAR: Failed
CHEERFUL GALLOPER: Failed
NEPTUNE: Success
MORNING LIGHT: Success
SON OF A GUN: Success
DOUBLE WIN: Success
EASY WIN: Success
COMMODORE: Success
FARRINGTON: Failed
KIDDY PAL: Failed
MONEY GALORE: Failed
IN LUCK: Success
SHAMROCK: Success
DRAGON HERO: Failed
THE BARBICAN: Failed
LUCKY LAD: Success
BAO TUNG: Failed
CARRY ON: Success
FRENCH CONNECTION: Success
MESSAGE: Success
FLAMME D'OR: Failed
NING PO: Failed
SPOILED BRAT: Failed
COUNTRY PARDNER: Failed
EMERALD FORTUNE: Success
SHARK FIGHTER: Failed
RASSELAS II: Failed
MASCOT:

DA VINCI: Success
MARKSMAN: Success
WINNING PHILIPPE: Failed
JACK-O'-LANTERN: Failed
FILLALINE: Failed
FLYING PHOENIX: Success
COVENANTER: Failed
VILLAGE BARD: Failed
STEELMASTER: Failed
LUCKY FLOWER: Failed
FOREVER VICTORY: Failed
SHALIMAR: Failed
KEEP IT UP: Success
COME FIRST: Failed
ALL HAPPY: Success
GOLDEN CHANCE: Success
GOSHAWK: Failed
EVER WIN: Failed
MARTINI: Failed
GLORY: Success
HAZEL TREE: Failed
TOP GAIN: Failed
HIGH SPEED: Success
GOLD STAR: Success
KANE KACHI: Failed
PROGRESSION: Success
CRACKERJACK: Success
PETE'S EARNINGS: Failed
RACING DEMON: Success
GRAND CHOICE: Success
AMBER STAR: Success
DAVIS SEA: Failed
SILVER JUBILEE: Failed
SMOKY: Failed
EL CID: Success
BIG SALOON: Failed
SILVER SEASON: Failed
ALWAYS KING: Failed
TWILIGHT: Success
WIN MORE: Success
MAZEL: Failed
BINGO: Success
MINI COMPUTER: Failed
FINEST HOUR: Success
WORLD SELECT: Failed
ROBIN HOOD: Success
MONA LISA: Failed
GOLDEN KEY: Success
TRY ME: Success
FAFA: Success
DOUBLE TOP: Success
ARC DE TRIOMP

WAH MAY LUCK: Success
E HONEY: Success
FOREVER: Success
SACRED GIANT: Success
OFF THE REEL: Success
FA FA: Success
OSCAR GLORY: Success
DRAGON FORTUNE: Success
THE ANOMALY: Success
PLEASANT ENDEAVOR: Success
CHARITY FUN: Success
LOYAL BABY: Success
LET'S DO IT: Failed
WHEN YOU BELIEVE: Success
HARMONY WIN WIN: Failed
A LA KING: Success
CASA KINGDOM: Failed
H Z TOURBILLON: Failed
WIN TECH: Success
NOBLE BOYZ: Failed
SUNNY BABY: Success
BREEDERS' DING: Failed
HARMONY FIRE: Success
EVERYONE'S VICTORY: Failed
RUN DES RUN: Success
HURRY HURRY GAIN: Failed
FLAGSHIP GLORY: Failed
INDICATIVE VOTE: Success
TURQUOISE ALPHA: Success
SUPER DYNAMITE: Success
MAJESTIC STAR: Failed
SPEND: Success
GOODMANSHIP: Success
ERNEST FEELING: Failed
DRAGON KINGDOM: Success
MULTIPOWER: Failed
FANTASTIC WAY: Failed
ETERNAL BEAUTY: Success
BAY TIBBS: Success
PARTERRE: Success
SMILING TIME: Failed
TRILLION WIN: Success
MOST WELCOME: Success
WAR WEAPON: Success
WHO LOVES THE ACE: Success
FANTASTIC TREASURE: Success

,Horse_ID,Horse_Name,Country,URL,URL_Country,Foal_Date,success_flag
0,1,SUPER AXIOM,AUS,https://www.racenet.com.au/profiles/horse/super-axiom,https://www.racenet.com.au/profiles/horse/super-axiom-aus,09/08/2017,1.0
1,2,SEA ELITE,AUS,https://www.racenet.com.au/profiles/horse/sea-elite,https://www.racenet.com.au/profiles/horse/sea-elite-aus,NaN,0.0
2,3,LITTLE PLAYER,NZ,https://www.racenet.com.au/profiles/horse/little-player,https://www.racenet.com.au/profiles/horse/little-player-nz,11/10/2016,1.0
3,4,NEW FUTURE,AUS,https://www.racenet.com.au/profiles/horse/new-future,https://www.racenet.com.au/profiles/horse/new-future-aus,12/10/2016,1.0
4,5,RED MAJESTY,NZ,https://www.racenet.com.au/profiles/horse/red-majesty,https://www.racenet.com.au/profiles/horse/red-majesty-nz,13/09/2016,1.0
...,...,...,...,...,...,...,...
13975,14140,SKY FOREVER,AUS,https://www.racenet.com.au/profiles/horse/sky-forever,https://www.racenet.com.au/profiles/horse/sky-forever-aus,21/09/2017,1.0
13976,14141,SHINING RED,IRE,https://www.racenet.com.au/profiles/horse/shining-red,https://www.racenet.com.au/profiles/horse/shining-red-ire,28/02/2018,1.0
13977,14142,NORDIC SKY,BRZ,https://www.racenet.com.au/profiles/horse/nordic-sky,https://www.racenet.com.au/profiles/horse/nordic-sky-brz,07/08/2017,1.0
13978,14143,HELENE ALLIBOR,AUS,https://www.racenet.com.au/profiles/horse/helene-allibor,https://www.racenet.com.au/profiles/horse/helene-allibor-aus,17/10/2017,1.0


#### Export to CSV:

In [37]:
path = r'C:\Users\USER\Documents\HKUST'

#horses_lookup.to_csv(path + "horse_foal_date.csv", sep='\t', encoding='utf-8', index=False)

horses_lookup.to_csv('horse_foal_date.csv', encoding='utf-8', index=False)